## Guia de Resolucion de Proyecto Integrador

### Objetivo
Poder ofrecer de guia y explicacion al trabajo integrador ofrecido por SoyHenry, desde ahora nombrado como la institucion, en el modulo 4 de Data Science, relacionado con las distintas herramientas de Big Data y auxiliares que permiten una introduccion al trabajo de grandes volumenes de datos mediante un equipo conjunto de trabajo desde un servidor remoto.

### Metodologia
Vamos a seguir paso a paso la resolucion de los ejercicios segun planteados, utilizando el mismo material que se encuentra en la carpeta '/DS-M4-Herramientas_Big_Data', aunqune en el transcurso del trabajo veremos como poder modificarla dentro del entorno Linux de nuestra maquina virtual.

### Pasos previos
Es necesario tener una maquina virtual de al menos 60 GB de memoria, que utilice Linux Ubuntu. En caso de tener la dada por la institucion, debemos incrementar el espacio de la misma siguiendo los pasos del archivo 'Resizing_VM.md'.
Ademas, para emular una conexion a un servidor remoto y para una mejor interaccion con la consola de la VM, vamos a utilizar Putty, cuya instalacion no vamos a explicar en este practico.

### Ejercicio 1: Entorno HDFS

Hadoop y, principalmente hdfs, es una herramienta que permite la accesibilidad y consistencia de la informacion dentro de un conjunto de servidores.

Una ves iniciada la maquina virtual con la nueva memoria de 60 GB, avanzaremos con el paso inicial... Para poder ejecutar los archivos dados, debemos descargarlos dentro de la maquina virtual, ya que la misma no tiene ninguna conexion con el CPU Windows o Mac que estemos utilizando.

Para realizar esto podemos hacerlo de dos formas, utilizando git o mediante WinSCP. Dado que WinSCP es una redundancia ya que habria que usar git de todas formas, dejaremos esta herramienta para una explicacion futura.


In [ ]:
git clone https://github.com/lopezdar222/herramientas_big_data # este codigo descarga la carpeta de trabajo en la VM
ls # atributo de listado, en este caso listar carpetas del directorio actual, verificar que exista 'DS-M4-Herramientas_Big_Data'
cd DS-M4-Herramientas_Big_Data # esto basicamente es el comando para ingresar a la carpeta homonima
sudo docker-compose -f docker-compose-v1.yml up -d 
# el comando docker-compose es para crear multiples contenedores a partir de un archivo yml, por defecto docker-compose.yml.
# agregamos -f para indicar el nombre del archivo a ejecutar y evitar el por defecto, y el -d para poder mantenernos en consola para seguir ejecutando codigo. 

Daremos una breve explicacion de lo ocurrido en este paso por unica vez al realizar un docker-compose para que se pueda entender este misterio... Para ello, ver el archivo docker-compose-v1.yml en Windows o en el mismo GitHub. <br>
Veremos que utilizando el docker-compose version 3, llamamos a describir diversos contenedores, cada uno con su imagen relacionada y otros atributos:<br>
<ul>
    <li>Namenode: este contenedor es el administrador de metadata y particiones del sistema de hadoop. El mismo tiene una imagen ya creada a utilizar y dos puertos de conexion (veremos que se conectan con varios contenedores). Ademas, cuenta con dos volumenes en formato hdfs (ya lo veremos) para poder resguardar la inforamcion en caso de reinicio o desaparicion del mismo dentro de la maquina virtual.</li>
    <li>Datanode: este nodo es quien almacena la informacion particionada que le brinda Namenode.. en este caso las particiones son todas sin redundancia por no tener nodos esclavos (o copias). Se conecta a Namenode mediante el puerto 9870 y en el entorno hdfs al namenode mediante el puerto 9000. Finalmente, tiene el puerto 9864 abrierto para conexiones hacia el mismo.</li>
    <li>Resourcemanager: es el contenedor que optimiza el uso de los datanodes, sirviendo como interactor entre Namenode y Datanode.</li>
    <li>Historyserver: tambien se conecta con Namenode y Datanode pero en este caso con el fin de llevar un historial de las operatorias de MapReduce, se utiliza para eficientizar operaciones.</li>
<br></ul>
Entonces, este docker-compose genera esta red de contenedores interconectados para su posterior utilizacion, en este caso dentro del entorno hadoop.

In [ ]:
sudo docker cp Datasets namenode:/home/ #copia la carpeta entera dentro del contenedor namenode, carpeta home
sudo docker exec -it namenode bash #ingresas al contenedor namenode (exec es para ejecutar un contenedor corriendo, -it para que sea iteractivo, namenode es el nombre del contenedor y bash es para decirle que queres trabajar sobre el bash del programa)
cd home #ingresas a la carpeta donde se copio Datasets
ls #muestra que hay en dicho directorio
hdfs dfs -mkdir -p /data #hdfs dfs te conecta con el entorno de archivos particionados hadoop, -mkdir es para crear un directorio, -p es para crear las carpetas madres necesarias y /data es el nombre del directorio
hdfs dfs -put /home/Datasets/* /data #en este caso estas particionando y generando redundancia (aunque no la haya por ser solo un contenedor de Datanode) a los archivos existentes en Datasets dentro del entorno hadoop
exit #para salir del contenedor y volver a nuestra carpeta dentro de la VM

Con este codigo, obtenemos los archivos en Datasets particionados y replicados segun indica la configuracion del contenedor namenode.

### Ejercicio 2: Hive

Hive es una herramienta que permite la gestion de bases de datos relacionales dentro del entorno de hadoop y archivos particionados.

En este paso aprenderemos a ejecutar un script Hive dentro de un contenedor, cuyo objetivo es crear tablas a partir de nuestros Datasets DENTRO DEL ENTORNO HDFS y trabajar sobre ellas.    

In [ ]:
sudo docker-compose -f docker-compose-v2.yml up -d #ejecutar el docker-compose-v2 que es un poco mas completo que el 1, pero con la misma sintaxis apra que puedas investigar usando ChatGPT
sudo docker cp Paso02.hql hive-server:/opt/Paso02.hql #copias el archivo dado para ejecutar las tablas del entorno hdfs dentro de hive
sudo docker exec -it hive-server bash #como hicimos en el punto anterior, nos metemos dentro del contenedor
hive -f Paso02.hql #dentro del contenedor, con este comando ejecutas el programa Hive y haces que corra el script Paso02.hql
exit #vuelve a la consola normal

### Ejecicio 3: Formatos de Almacenamiento

Ahora vamos a buscar convertir estas bases de dato a archivos con otros formatos de almacenamiento:

In [ ]:
sudo docker cp Paso03.hql hive-server:/opt/Paso03.hql #copiamos el archivo para el paso 3
sudo docker exec -it hive-server bash
ls #verificamos que este el archivo Paso03.hql
hive -f Paso03.hql
hive #para ejecutar el programa y asi ver las tablas creadas

In [ ]:
-- para poder ver las tablas creadas dentro de hive
show databases; --esto ya es codigo parecido a SQL, porque estas ejecutando codigo HQL dentro de Hive.
use integrador; --usamos la base de datos integrador
show tables; --para que nos muestre las tablas creadas
select * from calendario LIMIT 5; --para ver como esta esta tabla
select * from canal_venta LIMIT 5; --idem anterior
select * from cliente LIMIT 5; --idem anterior
select * from compra LIMIT 5; --idem anterior
select * from empleado LIMIT 5; --idem anterior
select * from gasto LIMIT 5; --idem anterior
select * from producto LIMIT 5; --idem anterior
select * from proveedor LIMIT 5; --idem anterior
select * from sucursal LIMIT 5; --idem anterior
select * from tipo_gasto LIMIT 5; --idem anterior
select * from venta LIMIT 5; --idem anterior

ATENCION: aca puede aparecer el primer error del integrador... si al final de la ejecucion dice Failed with exception java.io [...] /idtipodegasto=1/000000_0 returned false

Esto significa que ya intentaste hacer este paso y estas teniendo un problema por nomenclatura duplicada:

In [ ]:
exit
sudo docker exec -it namenode bash #ingresas al administrador de archivos
hdfs dfs -ls / #verificas si tenes la carpeta data2 creada
hdfs dfs -ls /data2/gasto/ #verificas si ya tenes las carpetas idtipogasto=1 creda y sucesivas
# de ser asi, seguis con lo siguiente
hdfs dfs -rm -r /data2/gasto/* #eliminando todas esas carpetas, -rm es para remover y -r es para indicar que sea recursivo hasta que elimine todos los archivos dentro de la carpeta y finalmente las carpetas
exit #para volver a la VM

#solucionado el error volvemos a intentarlo desde el inicio del ejercicio salteando el cp (copy)

ATENCION: tenemos un segundo error... que dice en este caso FAILED: SemanticException [Error 10044]: [...]

Esto es debido a que la tabla del datasets no se corresponde a la tabla creada en el script (diferente cantidad de columnas), lo corregimos y lo corremos manualmente (y terminamos de correr el script manual por facilidad, mas adelante veremos como modificar archivos):

In [ ]:
INSERT INTO calendario
SELECT
id,
fecha,
anio,
mes,
dia,
trimestre,
semana,
dia_nombre,
mes_nombre
FROM integrador.calendario;

DROP TABLE IF EXISTS proveedor;
CREATE EXTERNAL TABLE IF NOT EXISTS proveedor (
IDProveedor INTEGER,
Nombre VARCHAR(40),
Address VARCHAR(150),
City VARCHAR(80),
State VARCHAR(50),
Country VARCHAR(20),
departamen VARCHAR(50)
)
STORED AS PARQUET
LOCATION '/data2/proveedor'
TBLPROPERTIES ('parquet.compression'='SNAPPY');

INSERT INTO proveedor
SELECT
IdProveedor,
Nombre,
Direccion,
Ciudad,
Provincia,
Pais,
Departamento
FROM integrador.proveedor;

DROP TABLE trips_part;

-- No seguimos con las dos sentencias finales ya que no existe la base de datos asociada para cargar las particiones.

### Ejercicio 4: SQL e Indices

Si cerraste la sesion y te paso lo mismo que a mi, que las bases de dato en hive creadas en el punto anterior ya no existen, no te preocupes... vamos a aprovechar la situacion para aprender una herramienta adicional WinSCP.

Primero, vamos a realizar los cambios que vimos en el punto anterior sobre el archivo Paso03.hql dentro de nuestra PC Windows (si no lo descargaron, haganlo), esto quiere decir modificar la query de la linea 277 para que coincida con las columnas de la query de la linea 262; y elimnar las ultimas dos querys que no corrimos.

Una vez guardado el archivo, ejecutan el programa de Windows WinSCP, en cuyo caso les pedira que ingresen el hostname usual, asi como usuario y contrasena de la VM en cuestion. Finalmente aparecen en una pantalla partida, en donde en el lado izquierdo tienen el explorador de archivos de Windows y a la derecha, el de la VM. Luego, eliminan el Paso03.hql de la VM y copian el nuevo existente en Windows.

A partir de aqui, volvemos a correr los pasos 2 y 3 sobre la VM y no deberia haber ningun inconveniente, habiendo ademas aprendido a conectar ambos entornos de una manera sensilla para el intercambio de archivos.

A parir de ello, seguimos con el paso 4:

In [ ]:
sudo docker cp Paso04.hql hive-server:/opt/Paso04.hql
sudo docker exec -it hive-server bash
hive -f Paso04.hql

### Ejercicio 5: No-SQL

#### HBase:

